In [47]:
import os
import json
import torch
from torch.utils.data import DataLoader, Dataset
import clip
from PIL import Image
from tqdm import tqdm

# JSON 데이터 로드
json_file_path = r"E:\AI_HUB\data\Sublabel\SbL\extracted_data_all_caption.json"
with open(json_file_path, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# 최대 길이 설정 (CLIP 모델의 컨텍스트 제한)
MAX_CONTEXT_LENGTH = 77

def truncate_text(text, max_length=77):
    """
    입력 텍스트를 공백 단위로 나눠서 최대 길이에 맞게 자릅니다.
    """
    tokens = text.split()  # 공백 기준으로 텍스트를 나눕니다.
    if len(tokens) > max_length:
        tokens = tokens[:max_length]  # 최대 길이에 맞게 자르기
    return " ".join(tokens)  # 다시 문자열로 변환

# 이미지 데이터셋 클래스 정의
class TourismDataset(Dataset):
    def __init__(self, data, image_dir, preprocess):
        self.data = data
        self.image_dir = image_dir
        self.preprocess = preprocess

        # 이미지와 텍스트 매칭 데이터 생성
        self.samples = [
            (
                os.path.join(image_dir, entry["PHOTO_FILE_NM"]),
                truncate_text(entry["CAPTION"])  # 공백 기준 자르기
            )
            for entry in data
            if os.path.exists(os.path.join(image_dir, entry["PHOTO_FILE_NM"]))
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, keywords = self.samples[idx]
        image = self.preprocess(Image.open(image_path).convert("RGB"))
        return image, keywords

# CLIP 모델과 Preprocessor 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image_directory = r"E:\AI_HUB\data\Sublabel\SbL\photo"

# 데이터셋 생성 및 DataLoader 정의
dataset = TourismDataset(data, image_directory, preprocess)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 학습 루프
for batch_idx, (images, texts) in enumerate(tqdm(dataloader)):
    try:
        # 이미지를 GPU로 이동
        images = images.to(device)

        # 텍스트를 잘라주기 (공백 기준으로 토큰 제한)
        texts = [truncate_text(text, max_length=77) for text in texts]

        # CLIP의 토크나이저를 사용하여 텍스트를 토큰화
        texts = clip.tokenize(texts).to(device)

        # 이미지와 텍스트 특징 추출
        with torch.no_grad():
            image_features = model.encode_image(images)
            text_features = model.encode_text(texts)

        # 정규화
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # 유사도 계산
        similarity = (image_features @ text_features.T).cpu().numpy()
        print(f"Batch {batch_idx + 1} 유사도 매트릭스:\n", similarity)

    except Exception as e:
        print(f"에러 발생 (Batch {batch_idx + 1}): {e}")
        continue


  1%|          | 1/192 [00:06<20:04,  6.31s/it]

Batch 1 유사도 매트릭스:
 [[0.17020105 0.17129105 0.17733842 0.17619379 0.18143858 0.18230945
  0.18028206 0.17333719 0.17707315 0.18288758 0.18431638 0.17565092
  0.18411355 0.18237752 0.18496178 0.17897412]
 [0.19158983 0.19149037 0.19304098 0.1968408  0.19221082 0.20151448
  0.19716525 0.19107756 0.19363311 0.19929165 0.1856907  0.18759494
  0.19773772 0.19662893 0.19990723 0.19608682]
 [0.18061806 0.1796092  0.18624139 0.19643551 0.18115    0.19708642
  0.19342372 0.18472905 0.18983759 0.19279008 0.18099841 0.18430218
  0.18915775 0.19314033 0.20664299 0.18859376]
 [0.19783255 0.19889504 0.19965322 0.20548797 0.19667627 0.21122345
  0.20374973 0.20006439 0.19824201 0.2051678  0.20442162 0.19551131
  0.20614426 0.20566987 0.2142874  0.20052731]
 [0.17653345 0.17833738 0.17884193 0.18619439 0.1743839  0.18645924
  0.1821909  0.17875864 0.178839   0.18049374 0.17321876 0.17291574
  0.17964497 0.1806088  0.19578326 0.1765399 ]
 [0.18650794 0.1839263  0.18774068 0.19899441 0.19012392 0.2016653

  1%|          | 2/192 [00:12<19:25,  6.13s/it]

에러 발생 (Batch 2): Input 좌측에 건물들과 울타리와 돌들이 있고 아래쪽에 도로가 있고 우측에 울타리와 나무들이 있습니다 is too long for context length 77


  1%|          | 2/192 [00:15<24:44,  7.82s/it]


KeyboardInterrupt: 